## 🛒 Retail Trial Store Analysis

### 📌 Project Overview
This project evaluates the impact of a trial layout change in selected stores using SQL and Python.

### 🛠️ Tools Used
- Python (Pandas, Matplotlib, Seaborn, Scipy)
- Statistical Testing (T-Test)
- Data Visualization

### 🔍 Key Insights
✅ Trial Stores 86 & 88 showed **significant sales growth**  
❌ Trial Store 77 **did not show significant improvement**  
➡️ Business should **expand** the trial changes to stores similar to 86 & 88.

### 🚀 How to Run the Code
1. Clone the repository  
2. Install required libraries (`pip install pandas matplotlib seaborn scipy`)  
3. Run `quantium_trial_store_analysis.py`

### 📈 Visual Insights
- The sales trends of trial and control stores were **similar before the intervention**, confirming proper control store selection.
- **Trial Stores 86 & 88 showed clear spikes** post-intervention, which was confirmed statistically.
- **Trial Store 77 did not show a significant difference**, meaning the intervention **had no measurable impact there**.

### 📊 Business Recommendations
- **Expand changes to stores similar to 86 & 88**.
- **Reassess Store 77 and investigate other influencing factors.**
- **Monitor long-term impact before full rollout**.
  




1. Data Exploration & Preprocessing
We start by loading and inspecting the dataset. This helps understand the columns, missing values, and data types.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('QVI_data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.info()

Column Overview: The dataset contains 264,834 rows and 12 columns including important ones like STORE_NBR, TXN_ID, TOT_SALES, and LYLTY_CARD_NBR. We’ll focus on stores 77, 86, and 88 for our trial stores.

Data Cleaning: We'll ensure that the DATE column is in datetime format and handle any other necessary preprocessing steps (e.g., removing duplicates or handling missing values).

In [ ]:
# Convert 'DATE' to datetime format
df['DATE'] = pd.to_datetime(df['DATE'])

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
store_summary = df.groupby('STORE_NBR').agg(
    total_sales=('TOT_SALES', 'sum'),
    total_customers=('LYLTY_CARD_NBR', pd.Series.nunique),
    avg_transactions_per_customer=('TXN_ID', 'count')
).reset_index()

store_summary['avg_transactions_per_customer'] = store_summary['avg_transactions_per_customer'] / store_summary['total_customers']

store_summary.head()

In [ ]:
# Ensure STORE_NBR is the index
store_summary = store_summary.set_index('STORE_NBR')

# ✅ Compute correlation between stores (not metrics)
correlation_matrix = store_summary.T.corr()  # Transpose before correlation

# ✅ Print to check if STORE_NBR is now the index
correlation_matrix.head() # Should show store numbers in both index & columns

In [ ]:
trial_stores = [77, 86, 88]  # Trial stores

# Get correlations for trial stores
correlation_with_trials = correlation_matrix.loc[trial_stores]

# Find the most similar stores (excluding itself)
control_stores = correlation_with_trials.apply(lambda x: x.drop(trial_stores).abs().idxmax(), axis=1)

print("Selected Control Stores:", control_stores)


To assess the effectiveness of the trial, each trial store was paired with a control store based on similarity in historical sales patterns. The selected pairs are:  
TS     CS  
77     50  
86     70  
88     155


In [ ]:
import matplotlib.pyplot as plt

# Define trial and control store pairs
trial_control_pairs = {77: 50, 86: 70, 88: 155}  # Your selected control stores

# Loop through each pair and plot their comparison
for trial_store, control_store in trial_control_pairs.items():
    # Filter data for the trial and control stores
    trial_data = df[df['STORE_NBR'] == trial_store]  # Change to 'STORES' if needed
    control_data = df[df['STORE_NBR'] == control_store]

    # Aggregate total sales by date
    trial_sales = trial_data.groupby('DATE')['TOT_SALES'].sum()
    control_sales = control_data.groupby('DATE')['TOT_SALES'].sum()

    # Plot sales comparison
    plt.figure(figsize=(10, 5))
    plt.plot(trial_sales, label=f"Trial Store {trial_store}", color='blue')
    plt.plot(control_sales, label=f"Control Store {control_store}", color='red', linestyle='dashed')

    # Labels, title, and legend
    plt.xlabel("Date")
    plt.ylabel("Total Sales")
    plt.title(f"Sales Comparison: Trial Store {trial_store} vs Control Store {control_store}")
    plt.legend()
    plt.xticks(rotation=45)

    # Show plot
    plt.show()

In [ ]:
from scipy.stats import ttest_ind

# Dictionary to store test results
comparison_results = {}

for trial_store, control_store in trial_control_pairs.items():
    # Filter data
    trial_sales = df[df['STORE_NBR'] == trial_store].groupby('DATE')['TOT_SALES'].sum()
    control_sales = df[df['STORE_NBR'] == control_store].groupby('DATE')['TOT_SALES'].sum()

    # Perform T-test
    t_stat, p_value = ttest_ind(trial_sales, control_sales, equal_var=False)

    # Store results
    comparison_results[trial_store] = {
        'Control Store': control_store,
        'T-Statistic': t_stat,
        'P-Value': p_value
    }

    # Print results
    print(f"Trial Store {trial_store} vs Control Store {control_store}")
    print(f"T-Statistic: {t_stat:.4f}, P-Value: {p_value:.4f}")

    # Interpretation
    if p_value < 0.05:
        print(" Significant increase in sales (p < 0.05) ✅")
    else:
        print(" No significant increase in sales (p >= 0.05) ❌")

    print("-" * 50)

In [ ]:
# Calculate percentage growth in total sales for each trial store
sales_growth = {}  # Store growth for each store

for trial_store, control_store in trial_control_pairs.items():
    pre_trial_sales = df[(df['STORE_NBR'] == trial_store) & (df['DATE'] < '2019-01-01')]['TOT_SALES'].sum()
    post_trial_sales = df[(df['STORE_NBR'] == trial_store) & (df['DATE'] >= '2019-01-01')]['TOT_SALES'].sum()

    if pre_trial_sales > 0:
        sales_growth[trial_store] = ((post_trial_sales - pre_trial_sales) / pre_trial_sales) * 100
    else:
        sales_growth[trial_store] = "N/A (No sales before trial)"


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

for trial_store, control_store in trial_control_pairs.items():
    trial_customers = df[(df['STORE_NBR'] == trial_store) & (df['DATE'] >= '2019-02-01')].groupby('DATE')['LYLTY_CARD_NBR'].nunique()
    control_customers = df[(df['STORE_NBR'] == control_store) & (df['DATE'] >= '2019-02-01')].groupby('DATE')['LYLTY_CARD_NBR'].nunique()

    plt.plot(trial_customers, label=f"Trial Store {trial_store}", linestyle="-", linewidth=2)
    plt.plot(control_customers, label=f"Control Store {control_store}", linestyle="--", linewidth=2)

plt.xlabel("Date")
plt.ylabel("Unique Customers")
plt.title("Customer Growth Trends: Trial vs. Control Stores (Feb-May)")
plt.legend()
plt.xticks(rotation=45)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

for trial_store, control_store in trial_control_pairs.items():
    # Unique Customers (Customer Growth)
    trial_customers = df[(df['STORE_NBR'] == trial_store) & (df['DATE'] >= '2019-02-01')].groupby('DATE')['LYLTY_CARD_NBR'].nunique()
    control_customers = df[(df['STORE_NBR'] == control_store) & (df['DATE'] >= '2019-02-01')].groupby('DATE')['LYLTY_CARD_NBR'].nunique()

    plt.plot(trial_customers, label=f"Trial {trial_store} - Customers", linestyle="-", linewidth=2)
    plt.plot(control_customers, label=f"Control {control_store} - Customers", linestyle="--", linewidth=2)

plt.xlabel("Date")
plt.ylabel("Unique Customers")
plt.title("Customer & Sales Growth: Trial vs. Control Stores (Feb-May)")
plt.legend()
plt.xticks(rotation=45)
plt.show()


In [ ]:


# Filter data for February to May
df = df[(df["DATE"] >= "2019-02-01") & (df["DATE"] <= "2019-05-31")]

# Group by date and store to get customer counts
customer_counts = df.groupby(["DATE", "STORE_NBR"])["LYLTY_CARD_NBR"].nunique().unstack()

# Plot the trends
plt.figure(figsize=(10, 5))
plt.plot(customer_counts.index, customer_counts[77], label="Trial 77 - Customers", color="blue")
plt.plot(customer_counts.index, customer_counts[86], label="Trial 86 - Customers", color="green")
plt.plot(customer_counts.index, customer_counts[88], label="Trial 88 - Customers", color="purple")

# Formatting
plt.title("Customer Growth in Trial Stores (Feb - May)")
plt.xlabel("Date")
plt.ylabel("Unique Customers")
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)

# Show plot
plt.show()


In [ ]:
print("\n--- Final Summary ---")
for trial_store, control_store in trial_control_pairs.items():
    t_stat = comparison_results[trial_store]['T-Statistic']
    p_value = comparison_results[trial_store]['P-Value']

    print(f"🛒 Trial Store {trial_store} vs Control Store {control_store}")
    print(f"📊 Sales Growth: {sales_growth[trial_store]}%")  # Now prints correct growth per store
    print(f"📈 T-Statistic: {t_stat:.4f}, P-Value: {p_value:.4f}")

    if p_value < 0.05:
        print("✅ Significant Increase in Sales\n")
    else:
        print("❌ No Significant Increase in Sales\n")

**Insights from the Visualizations**  
**Sales Trends Over Time:**   
* The sales patterns of trial and control stores were quite similar before the intervention, confirming that the control stores were well-matched.  
* After the intervention, some trial stores show noticeable spikes, indicating a potential impact of the changes made.  

**Trial Store 77 vs Control Store 50:**   
* The sales trends of both stores remain quite close throughout the period.  
* No significant increase in sales is observed after the intervention.  
**Insight:** The changes made in Trial Store 77 did not have a measurable impact on sales.  

**Trial Store 86 vs Control Store 70:**   
* The trial store generally shows higher peaks compared to the control store after the intervention.  
* The t-test confirms a statistically significant sales increase.  
**Insight:** The changes in Trial Store 86 had a positive impact on sales. The company should analyze what worked and consider implementing it in more stores.  

**Trial Store 88 vs Control Store 155:**   
* The trial store shows a clear upward trend compared to the control store after the intervention.  
* The t-test result strongly confirms a significant increase in sales.  
**Insight:** The intervention in Trial Store 88 was highly successful. This store experienced the highest impact, suggesting the strategy should be replicated in similar stores.  


---



**Overall Recommendations**  
* Expand the changes applied in Stores 86 and 88 to other locations.
* Re-evaluate the strategy used in Store 77, as it did not produce a significant effect.
* **Further Analysis:** Identify what specifically worked in Stores 86 and 88 (e.g., promotions, layout changes) and optimize the approach before scaling up.

**Business Recommendations**

Based on the analysis, the following recommendations are made:

* Expand the changes implemented in Stores 86 and 88 to similar stores, as they showed a significant increase in sales.
* Investigate Store 77 further, as the trial did not result in a meaningful impact.
* Monitor long-term impact in Stores 86 and 88 to ensure sustained growth before a full rollout.
* Consider additional factors such as product category performance and regional differences before full implementation.





**Conclusion**

The trial successfully improved sales in Stores 86 and 88, while Store 77 did not show a significant effect.  
The findings suggest that the changes should be rolled out strategically, focusing on stores similar to 86 and 88 while further analyzing factors influencing Store 77.

                                                                      THANK YOU